## OpenCV (2)
### 웹캠 얼굴 식별 & 개인정보 비식별화(blurring)

In [17]:
import cv2
import mediapipe as mp
import numpy as np
from facial_landmarks import FaceLandmarks
# Load face landmarks
fl = FaceLandmarks()

ModuleNotFoundError: No module named 'cv2'

In [12]:
import cv2

# Load the pre-trained face detection model from OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize the webcam
cap = cv2.VideoCapture(0)

while True:
  # Read a frame from the webcam
  ret, frame = cap.read()

  # Convert the frame to grayscale for face detection
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

  # Detect faces in the grayscale frame
  faces = face_cascade.detectMultiScale(gray, 1.1, 4)

  # Iterate over the detected faces
  for (x, y, w, h) in faces:
    # Blur the face area
    face_roi = frame[y:y+h, x:x+w]
    blurred_face = cv2.GaussianBlur(face_roi, (99, 99), 30)
    frame[y:y+h, x:x+w] = blurred_face

  # Display the frame with blurred faces
  cv2.imshow('Face Blurring', frame)

  # Break the loop if the 'q' key is pressed
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

# Release the webcam and destroy all windows
cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'cv2'

In [7]:
!pip3 install imutils numpy setuptools opencv-python

  Using cached imutils-0.5.4.tar.gz (17 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numpy-2.1.1-cp312-cp312-macosx_10_9_x86_64.whl.metadata (60 kB)
  Using cached setuptools-75.1.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached opencv-python-4.10.0.84.tar.gz (95.1 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
ERROR: Exception:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pip/_internal/cli/base_command.py", line 105, in _run_wrapper
    status = _inner_run()
             ^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pip/_internal/cli/base_command.py", line 96, in _inner_run
    return self.run(options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12

In [9]:
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import imutils
import time
import cv2
import os
import urllib

ModuleNotFoundError: No module named 'imutils'

In [6]:

# Define URLs and filenames for both files
prototxt_url = "https://raw.githubusercontent.com/dscoool/opencv/refs/heads/main/deploy.prototxt.txt"
prototxt_filename = "deploy.prototxt.txt"

caffemodel_url = "https://github.com/dscoool/opencv/raw/refs/heads/main/weights.caffemodel"
caffemodel_filename = "weights.caffemodel"

# Download the files if they don't exist locally
if not os.path.exists(prototxt_filename):
    urllib.request.urlretrieve(prototxt_url, prototxt_filename)

if not os.path.exists(caffemodel_filename):
    urllib.request.urlretrieve(caffemodel_url, caffemodel_filename)

# Load the model
detector = cv2.dnn.readNetFromCaffe(prototxt_filename, caffemodel_filename)

In [7]:
# offline version
# protoPath = "deploy.prototxt"
# modelPath = "res10_300x300_ssd_iter_140000.caffemodel"
# detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

### face detection


In [9]:
vs = VideoStream(src=0).start()
time.sleep(2.0)

fps = FPS().start()

while True:
    frame = vs.read()
    frame = imutils.resize(frame,width=600)
    (h, w) = frame.shape[:2]

    imageBlob = cv2.dnn.blobFromImage(cv2.resize(frame,(300,300)),1.0, (300,300),(104.0,177.0,123.0), swapRB=False, crop=False)

    detector.setInput(imageBlob)
    detections = detector.forward()

    for i in range(0,detections.shape[2]):
        confidence = detections[0,0,i,2]

        if confidence > 0.5:
            box = detections[0,0,i,3:7] * np.array([w,h,w,h])
            (startX, startY, endX, endY) = box.astype("int")

            face = frame[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]

            if fW < 20 or fH < 20:
                continue

            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,(96, 96), (0, 0, 0), swapRB=True, crop=False)

            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(frame, (startX, startY), (endX, endY),
                (0, 0, 255), 2)

    fps.update()
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
fps.stop()
cv2.destroyAllWindows()
vs.stop()

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
face_image = cv2.GaussianBlur(face,(99,99), 30)
            frame[startY:endY, startX:endX] = face_image

In [ ]:
import cv2
import numpy as np
import time
import argparse
import urllib



# Download the prototxt and caffemodel files
prototxt_url = 'https://raw.githubusercontent.com/dscoool/opencv/refs/heads/main/deploy.prototxt.txt'
caffemodel_url = 'https://github.com/dscoool/opencv/raw/refs/heads/main/weights.caffemodel'

# Create temporary files to store the downloaded content
prototxt_file = 'deploy.prototxt.txt'  # Temporary file for prototxt
caffemodel_file = 'weights.caffemodel' # Temporary file for caffemodel

# Download and save prototxt
urllib.request.urlretrieve(prototxt_url, prototxt_file)
# Download and save caffemodel
urllib.request.urlretrieve(caffemodel_url, caffemodel_file)

CONFIDENCE = 0.5

model = cv2.dnn.readNetFromCaffe(prototxt_file, caffemodel_file)

def face_pixelate(image,blocks=3):
    (h,w)=image.shape[:2]
    Xsteps=np.linspace(0,w,blocks+1,dtype='int')
    Ysteps=np.linspace(0,h,blocks+1,dtype='int')
    for i in range(1,len(Ysteps)):
        for j in range(1,len(Xsteps)):
            startX=Xsteps[j-1]
            endX=Xsteps[j]
            startY=Ysteps[i-1]
            endY=Ysteps[i]
            roi=image[startY:endY,startX:endX]
            (B,G,R)=[int(x) for x in cv2.mean(roi)[:3]]
            res=cv2.rectangle(image,(startX,startY),(endX,endY),(B,G,R),-1)
    return image

def face_blurring(image,blur_type,blocks=3):
    img_copy=np.copy(image)
    (h, w)=img_copy.shape[:2]
    blob=cv2.dnn.blobFromImage(cv2.resize(img_copy,(300,300)),1.0,(300,300),(104.0,177.0,123.0))
    model.setInput(blob)
    detections=model.forward()
    for i in range(0,detections.shape[2]):
        confidence=detections[0, 0, i, 2]
        if confidence>CONFIDENCE:
            box=detections[0, 0, i, 3:7]*np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            face=img_copy[startY:endY,startX:endX]
            if(blur_type=="pixelate"):
                face=face_pixelate(face,blocks)
            else:
                face=cv2.blur(face,(23, 23))
                img_copy[startY:endY,startX:endX]=face
    return img_copy


image_url = "https://raw.githubusercontent.com/dscoool/opencv/refs/heads/main/billgates.png"

if __name__=='__main__':
    ap = argparse.ArgumentParser()
    ap.add_argument("image", required=True, help="path to input image")
    ap.add_argument("output", required=True, help="path to output image")
    ap.add_argument("blur_type",type=str, default="simple",choices=["simple", "pixelated"],
	help="face blurring/anonymizing method")
    ap.add_argument("blocks", type=int, default=15, help="# of blocks for the pixelated blurring method")
    args = vars(ap.parse_args())
    img = cv2.imread(args.image)
    start_time = time.time()
    output=face_blurring(img,ap.blur_type,ap.blocks)
    end_time = time.time()
    t = end_time-start_time
    print('time: {0}s'.format(t))
    cv2.imwrite(args.output, output)

TypeError: 'required' is an invalid argument for positionals

Ref: https://github.com/Sudarshana2000/Automated_face_blurring
